# Prototype for D3D input files

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [22]:
import xarray as xr

## READ mdf files

In [3]:
mdf = pd.read_csv('/Users/brey/GitHub/pyPoseidon/pyPoseidon/misc/default.mdf',sep='=')

In [4]:
mdf=mdf.set_index(mdf.columns[0])

In [5]:
mdf.iloc[1]

 #Delft3D-FLOW  .03.02 3.42.00.17210#     #med.grd#
Name: Filcco , dtype: object

### sort out the spaces issue

In [6]:
mdfidx = mdf.index.str.strip()

In [7]:
'FlNcdf' in mdfidx

True

In [9]:
mdf = pd.read_csv('/Users/brey/Downloads/EUR/D3D/20100201.00/eur01.mdf',sep='=')

In [10]:
mdf = mdf.set_index(mdf.columns[0])

In [11]:
mdf.index = mdf.index.str.strip()

In [13]:
mdf

,#Delft3D-FLOW .03.02 3.42.00.17210#
Ident,
Commnt,
Filcco,#eur01.grd#
Filgrd,#eur01.enc#
MNKmax,771 516 1
Thick,100.0
Fildep,#eur01.dep#
Itdate,#2010-02-01#
Tunit,#M#
Tstart,0


### modify 

In [14]:
mdf.loc[mdf.index.str.contains('Itdate')]='#2016-08-20#'

In [16]:
mdf.loc[mdf.index.str.contains('Itdate')]

,#Delft3D-FLOW .03.02 3.42.00.17210#
Ident,
Itdate,#2016-08-20#


In [17]:
if not 'FlNcdf' in mdf.index : mdf.loc['FlNcdf'] = '#map his#'

### save

In [47]:
mdf.to_csv('/Users/brey/Downloads/test.mdf',sep='=')

##  GRID file

### Create xarray for grid

In [18]:
minlon= -79. 
maxlon=  -76.
minlat= 17.
maxlat= 19.
resolution=0.2

In [19]:
ni=int(round((maxlon-minlon)/resolution)) #these are cell numbers
nj=int(round((maxlat-minlat)/resolution))
  
maxlon=minlon+ni*resolution #adjust max lon to much the grid
maxlat=minlat+nj*resolution

       
ni=ni+1 # transfrom to grid points
nj=nj+1


In [20]:
x=np.linspace(minlon,maxlon,ni)
y=np.linspace(minlat,maxlat,nj)
gx,gy=np.meshgrid(x,y)


In [23]:
g = xr.Dataset({'lons': (['x', 'y'], gx),   
                'lats': (['x', 'y'], gy)})

In [24]:
g.attrs = {'Coordinate System': 'Spherical', 'alfori': 0.0, 'xori': 0.0, 'yori': 0.0}

In [25]:
g

<xarray.Dataset>
Dimensions:  (x: 11, y: 16)
Dimensions without coordinates: x, y
Data variables:
    lons     (x, y) float64 -79.0 -78.8 -78.6 -78.4 -78.2 -78.0 -77.8 -77.6 ...
    lats     (x, y) float64 17.0 17.0 17.0 17.0 17.0 17.0 17.0 17.0 17.0 ...
Attributes:
    Coordinate System:  Spherical
    xori:               0.0
    yori:               0.0
    alfori:             0.0

### write file

In [26]:
filename = '/Users/brey/Downloads/d3d.grd'

In [27]:
with open(filename,'w') as f:
    f.write('Coordinate System= {}\n'.format(g.attrs['Coordinate System']))
    f.write('{} {}\n'.format(g.lons.shape[1],g.lons.shape[0]))
    f.write('{} {} {}\n'.format(g.attrs['xori'],g.attrs['yori'],g.attrs['alfori']))
    for i in range(g.lons.shape[0]):
        f.write('ETA=  {} '.format(i+1))
        f.write(' '.join(map(str, g.lons[i,:].values)))
        f.write('\n')
    for i in range(g.lats.shape[0]):
        f.write('ETA=  {} '.format(i+1))
        f.write(' '.join(map(str, g.lats[i,:].values)))
        f.write('\n')

    

### Read grid file

In [28]:
filename = '/Users/brey/Downloads/d3d.grd'

In [29]:
header=pd.read_csv(filename,nrows=3,header=None,comment='*')
header

,0
0,Coordinate System= Spherical
1,16 11
2,0.0 0.0 0.0


In [30]:
cs = header.loc[0,0].split('=')[1].strip()
ni,nj = header.loc[1,0].split(' ')
ni,nj = int(ni),int(nj)
alfori, xori, yori = header.loc[2,0].split(' ')

In [31]:
d = pd.read_csv(filename,header=2,comment='*',delim_whitespace=True,engine='python',na_values='ETA=')

In [32]:
d = d.reset_index()

In [33]:
data = d.values[~np.isnan(d.values)]

In [34]:
data=np.array(data)

In [35]:
data = data.reshape(2,nj,ni+1) # including the row index

In [36]:
#clean up the row index
data = data[:,:,1:]

In [37]:
lons=data[0,:,:]
lats=data[1,:,:]

In [38]:
g = xr.Dataset({'lons': (['x', 'y'], lons),   
                'lats': (['x', 'y'], lats)})

In [39]:
g.attrs = {'Coordinate System': cs, 'alfori': alfori, 'xori': xori, 'yori': yori}

In [40]:
g

<xarray.Dataset>
Dimensions:  (x: 11, y: 16)
Dimensions without coordinates: x, y
Data variables:
    lons     (x, y) float64 -79.0 -78.8 -78.6 -78.4 -78.2 -78.0 -77.8 -77.6 ...
    lats     (x, y) float64 17.0 17.0 17.0 17.0 17.0 17.0 17.0 17.0 17.0 ...
Attributes:
    Coordinate System:  Spherical
    xori:               0.0
    yori:               0.0
    alfori:             0.0

In [41]:
g.lons

<xarray.DataArray 'lons' (x: 11, y: 16)>
array([[-79. , -78.8, -78.6, -78.4, -78.2, -78. , -77.8, -77.6, -77.4, -77.2,
        -77. , -76.8, -76.6, -76.4, -76.2, -76. ],
       [-79. , -78.8, -78.6, -78.4, -78.2, -78. , -77.8, -77.6, -77.4, -77.2,
        -77. , -76.8, -76.6, -76.4, -76.2, -76. ],
       [-79. , -78.8, -78.6, -78.4, -78.2, -78. , -77.8, -77.6, -77.4, -77.2,
        -77. , -76.8, -76.6, -76.4, -76.2, -76. ],
       [-79. , -78.8, -78.6, -78.4, -78.2, -78. , -77.8, -77.6, -77.4, -77.2,
        -77. , -76.8, -76.6, -76.4, -76.2, -76. ],
       [-79. , -78.8, -78.6, -78.4, -78.2, -78. , -77.8, -77.6, -77.4, -77.2,
        -77. , -76.8, -76.6, -76.4, -76.2, -76. ],
       [-79. , -78.8, -78.6, -78.4, -78.2, -78. , -77.8, -77.6, -77.4, -77.2,
        -77. , -76.8, -76.6, -76.4, -76.2, -76. ],
       [-79. , -78.8, -78.6, -78.4, -78.2, -78. , -77.8, -77.6, -77.4, -77.2,
        -77. , -76.8, -76.6, -76.4, -76.2, -76. ],
       [-79. , -78.8, -78.6, -78.4, -78.2, -78. , -77.8,

### Depth file

In [42]:
filename = '/Users/brey/Downloads/d3d.dep'

In [43]:
de = np.random.rand(g.lons.values.shape[0],g.lons.values.shape[1])

In [44]:
de[-1,:]=np.nan

In [45]:
de[np.isnan(de)]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan])

In [46]:
np.savetxt(filename, de)

In [47]:
np.loadtxt(filename)

array([[8.53369706e-01, 6.37855812e-01, 7.32094030e-01, 9.49404568e-01,
        4.02763067e-01, 7.57651294e-01, 4.31412885e-01, 4.21823416e-01,
        9.90949861e-01, 1.13694384e-01, 3.91249932e-02, 5.31019097e-01,
        8.41419629e-01, 8.89873448e-01, 3.59228465e-01, 4.32567603e-01],
       [3.59444109e-01, 4.09617319e-01, 9.03891193e-02, 8.80864198e-03,
        3.08417858e-01, 5.58908168e-01, 3.95580152e-01, 7.43260161e-01,
        5.04488214e-01, 2.72692386e-01, 8.96610669e-01, 7.51449001e-01,
        7.11316109e-01, 1.87904419e-01, 4.70664195e-01, 2.58239908e-01],
       [1.86567693e-01, 1.34459919e-01, 4.45432164e-02, 4.48079929e-01,
        3.13062764e-01, 9.44692018e-01, 6.23370706e-01, 4.03753270e-01,
        2.59382561e-01, 4.40019956e-01, 3.28318115e-01, 8.17279042e-01,
        5.91331181e-02, 6.18509983e-02, 1.94763810e-01, 3.91905476e-01],
       [5.24792503e-01, 4.02191939e-02, 7.21143802e-04, 8.34551504e-01,
        7.23523154e-01, 5.77731533e-01, 9.42634113e-01, 4.772